In [1]:
import cv2
import tensorflow as tf
import os 

# class_names = {0: '0', 1: '1', 2: '2', 3: '3', 4: '4',
#                5: '5', 6: '6', 7: '7', 8: '8', 9: '9',
#                10: 'a', 11: 'b', 12: 'c', 13: 'd', 14: 'e',
#                15: 'f', 16: 'g', 17: 'h', 18: 'i', 19: 'k',
#                20: 'l', 21: 'm', 22: 'n', 23: 'o', 24: 'p',
#                25: 'q', 26: 'r', 27: 's', 28: 't', 29: 'u',
#                30: 'v', 31: 'w', 32: 'x', 33: 'y'}

class_names = {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 
               9: 'k', 10: 'l', 11: 'm', 12: 'n', 13: 'o', 14: 'p', 15: 'q', 16: 'r', 
               17: 's', 18: 't', 19: 'u', 20: 'v', 21: 'w', 22: 'x', 23: 'y'}

In [2]:
# code to stack windows horizontally or vertically 
# taken from https://youtu.be/WQeoO7MI0Bs also can refer to the link in the video description

def stackImages(scale,imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        ver = hor
    return ver


In [ ]:
# to be used to reset the GPU if required

# from numba import cuda 
# device = cuda.get_current_device()
# device.reset()


In [10]:
from tensorflow.keras.models import load_model

model = load_model('model_mobilenetV2_(224-224)_Adam_68-72_(200-1100-500-150-24).h5')

model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [13]:
import numpy as np

cap = cv2.VideoCapture(1)  # 0 for laptop webcam and 1 for external webcam
flag=True

while flag:
    suc, img = cap.read()

    width = 224
    height = 224
    dim = (width, height)

# #    resize image
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

#     cv2.rectangle(img,(20,0),(260,240),(0,255,0),3)

#     cv2.rectangle(img,(20,0),(480,240),(0,255,0),3)
    
    img_array = tf.keras.preprocessing.image.img_to_array(resized)
    img_array = tf.expand_dims(img_array, 0)   # Create a batch
    
    with tf.device('/CPU:0'):
        predictions = model.predict(img_array)
        score = tf.nn.softmax(predictions[0])
       # print(score)

    cv2.putText(img, class_names[np.argmax(score)], (50,50), cv2.FONT_HERSHEY_SIMPLEX , 1, (0,0,255), 2, cv2.LINE_AA)

    imgStack = stackImages(0.6,[img,resized])
    cv2.imshow("Stacked Images", imgStack)   # displaying the stacked images

    key = cv2.waitKey(1)
    if key == ord('z'):  # if "z" is pressed the close the capture proccess
        flag=False
        cap.release()


cv2.destroyAllWindows()  # destroy/close the windows opened by the program